In [179]:
import os
import pandas as pd
import numpy as np

from new_functions import get_data, group_activities, calc_emissions, efs

output_file = 'WCports_results_23Mar.xlsx'
scenarios = ['WCports_test_ctv6', 'WCports_test_ctv8', 'WCports_test_sov6']

savedir = 'new/'

## percent of transit time to bin as maneuvering
perc_m = 0.1

strategies = []
ports_headers = []

for s in scenarios:
    times_df = get_data(s, output_file)
    grouped_df, ports = group_activities(times_df, perc_m)
    ports_headers += ports
    emissions_df = calc_emissions(grouped_df, efs, ports)
    strategies.append(emissions_df)

    #filename = s + '_emissions.xlsx'
    #savepath = savedir + filename
    #emissions_df.to_excel(savepath)

In [180]:
activities = ['idle at port', 'idle at site', 'maneuvering', 'transit']
adjusted = []

for df in strategies:
        
    if 'ctv' in df.values:
        df_copy = pd.DataFrame().reindex_like(df.head(4))
        df_copy['Vessel'] = df_copy['Vessel'].fillna('sov')
        df_copy['Activity'] = activities     
    elif 'sov' in df.values:
        df_copy = pd.DataFrame().reindex_like(df.head(4))
        df_copy['Vessel'] = df_copy['Vessel'].fillna('ctv')
        df_copy['Activity'] = activities
    
    df = pd.concat([df, df_copy], axis=0)
    df.set_index(['Vessel', 'Activity'], drop=True, inplace=True)
    df.fillna(0, inplace=True)
    df.sort_index(ascending=True, inplace=True)
    
    for col in df:
        if col in ports_headers:
            df.drop(col, axis=1, inplace=True)
            
    df.drop('Emissions_factor', axis=1, inplace=True)
    
    adjusted.append(df)

In [181]:
df_0 = adjusted[0]
df_1 = adjusted[1]
df_2 = adjusted[2]

In [182]:
for df, name in zip(adjusted, scenarios):
    savepath = 'new/testers/' + name + '.xlsx'
    df.to_excel(savepath)
    
joined_df = adjusted[0].join(adjusted[1:], lsuffix='_caller', rsuffix='_other')

joined_df.to_excel('new/testers/joined_test.xlsx')

In [183]:
scenarios

['WCports_test_ctv6', 'WCports_test_ctv8', 'WCports_test_sov6']